In [348]:
import pandas as pd
import numpy as np
import regex as re
df1 = pd.read_pickle('2013-2023_plays.pkl')



In [349]:
aaa = [
    'game_id',
    'play_id',
    'down',
    'posteam',
    'defteam',
    'home_team',
    'away_team',
    'posteam_score',
    'defteam_score',
    'posteam_type',
    'yardline_100',
    'goal_to_go',
    'posteam_timeouts_remaining',
    'defteam_timeouts_remaining',
    'score_differential', #Score differential between the posteam and defteam at the start of the play.
    'score_differential_post',#Score differential between the posteam and defteam at the end of the play.
    'vegas_wp',
    'home_wp_post',
    'away_wp_post',
    'fourth_down_failed',
    'third_down_failed',
    'cp',
    'complete_pass',
    'desc',
    'field_goal_attempt',
    'field_goal_result',
    'first_down',
    'first_down_pass',
    'first_down_penalty',
    'first_down_rush',
    'fourth_down_converted',
    'fourth_down_failed',
    'game_date',
    'game_half',
    'game_seconds_remaining',
    'half_seconds_remaining',
    'game_stadium',
    'home_opening_kickoff',
    'incomplete_pass',
    'interception',
    'kick_distance',
    'location',
    'no_score_prob',
    'pass',
    'pass_attempt',
    'pass_length',
    'pass_location',
    'pass_touchdown',
    'passing_yards',
    'penalty',
    'penalty_team',
    'play_type',
    'punt_attempt',
    'punt_blocked',
    'punter_player_id',
    'roof',
    'rush',
    'run_location',
    'sack',
    'score_differential',
    'season',
    'season_type',
    'spread_line',
    'stadium_id',
    'success',
    'td_team',
    'total_away_score',
    'total_home_score',
    'total_line',
    'touchdown',
    'vegas_wp',
    'weather',
    'wind',
    'wp',
    'xpass',
    'ydstogo',
    'yrdln',
    'drive',
    'drive_play_count',
    'wpa',
    'kicker_player_id'
]

In [350]:
df= df1[aaa]

In [351]:
df = df[df['weather'].notna()]
# keep only games with weather data


df_fg = df[df['field_goal_attempt']==1]

# only looking at FG data 

df_fg = df_fg[['game_date','weather','field_goal_result','kick_distance','kicker_player_id',]]

df_fg['result'] = np.where(df_fg['field_goal_result'] == 'made', 1, 0)
df_fg.drop('field_goal_result',axis=1,inplace=True)


# extract weather data
regex_pattern = r'^(?P<conditions>\w+)\sTemp:\s(?P<temp>\d+)°\sF,\sHumidity:\s\d+%,\sWind:\s\w+\s(?P<wind>\d+)\smph'

default_temp = 68
default_wind = 0

df_fg[['conditions', 'temp', 'wind']] = df_fg['weather'].str.extract(regex_pattern)
df_fg['conditions'].fillna('indoor',inplace=True)
df_fg['wind'].fillna(0,inplace=True)
df_fg['wind'].replace('', default_wind, inplace=True)

df_fg['temp'].fillna(default_temp,inplace=True)
df_fg['temp'].replace('', default_temp, inplace=True)


df_fg['temp'] = df_fg['temp'].astype(float)
df_fg['wind'] = df_fg['wind'].astype(float)
df_fg.drop('weather',axis=1,inplace=True)

In [352]:
df_fg[df_fg['result']==1]

,game_date,kick_distance,kicker_player_id,result,conditions,temp,wind
36315,2013-12-08,33.0,00-0026968,1,Snow,30.0,10.0
36394,2013-12-08,50.0,00-0028787,1,Snow,30.0,10.0
47174,2014-01-11,38.0,00-0025944,1,Rain,48.0,20.0
47193,2014-01-11,49.0,00-0025944,1,Rain,48.0,20.0
47237,2014-01-11,26.0,00-0025944,1,Rain,48.0,20.0
...,...,...,...,...,...,...,...
529798,2024-02-11,57.0,00-0033303,1,Sunny,54.0,6.0
529838,2024-02-11,24.0,00-0033303,1,Sunny,54.0,6.0
529846,2024-02-11,53.0,00-0038562,1,Sunny,54.0,6.0
529860,2024-02-11,29.0,00-0033303,1,Sunny,54.0,6.0


In [353]:
first_x = []
df_filtered = df_fg

for i in range(1, 345):  # Start at 1 to avoid 0 kicks
    # Calculate first 'i' kicks for each kicker
    df_first_i_kicks = df_filtered.groupby('kicker_player_id').head(i)

    # Calculate the accuracy for each kicker over their first 'i' kicks
    kicker_accuracy = df_first_i_kicks.groupby('kicker_player_id')['result'].agg(
        total_kicks='size',  # Count the total number of kicks
        made_kicks='sum',  # Sum up the 1s in the result to get successful kicks
        accuracy='mean'  # Calculate mean to get the accuracy
    ).reset_index()

    # Calculate the overall accuracy across all these kickers
    overall_accuracy = kicker_accuracy['made_kicks'].sum() / kicker_accuracy['total_kicks'].sum() if kicker_accuracy['total_kicks'].sum() > 0 else 0

    # Store the results with kick count and accuracy
    first_x.append({'position': i, 'accuracy': overall_accuracy})

# Print the result to check for result in first_x
# for result in first_x:
#     print(result)

first_x

[{'position': 1, 'accuracy': 0.7628865979381443},
 {'position': 2, 'accuracy': 0.7754010695187166},
 {'position': 3, 'accuracy': 0.7956204379562044},
 {'position': 4, 'accuracy': 0.8027777777777778},
 {'position': 5, 'accuracy': 0.7995495495495496},
 {'position': 6, 'accuracy': 0.819047619047619},
 {'position': 7, 'accuracy': 0.8214876033057851},
 {'position': 8, 'accuracy': 0.8218978102189781},
 {'position': 9, 'accuracy': 0.8183006535947712},
 {'position': 10, 'accuracy': 0.8222748815165877},
 {'position': 11, 'accuracy': 0.8186753528773073},
 {'position': 12, 'accuracy': 0.8176352705410822},
 {'position': 13, 'accuracy': 0.8171641791044776},
 {'position': 14, 'accuracy': 0.8190559440559441},
 {'position': 15, 'accuracy': 0.8220757825370676},
 {'position': 16, 'accuracy': 0.8230709275136399},
 {'position': 17, 'accuracy': 0.8217455621301775},
 {'position': 18, 'accuracy': 0.8198451794510908},
 {'position': 19, 'accuracy': 0.8181208053691276},
 {'position': 20, 'accuracy': 0.819640564

In [354]:
df_fg['kick_number'] = df_fg.sort_values(by='game_date').groupby('kicker_player_id').cumcount() + 1

accuracy_dict = {i + 1: d['accuracy'] for i, d in enumerate(first_x)}

df_fg['first_x'] = df_fg['kick_number'].map(accuracy_dict)

df_fg['real_accuracy'] = df_fg.sort_values(by='game_date').groupby('kicker_player_id')['result'] \
    .apply(lambda x: x.expanding().mean().shift(1)).fillna(0).reset_index(level=0, drop=True)


df_fg['weighted_accuracy'] = np.where(
    df_fg['kick_number'] == 1,
    df_fg['first_x'],
    np.where(
        df_fg['kick_number'] <= 100,
        df_fg['first_x'] * (1 - (3 * df_fg['kick_number'] / 100)) + df_fg['real_accuracy'] * (3 * df_fg['kick_number'] / 100),
        df_fg['real_accuracy']
    )
)

average_distance = df_fg['kick_distance'].mean()

df_fg['avg_prev_distance'] = df_fg.sort_values(by='game_date').groupby('kicker_player_id')['kick_distance'] \
    .apply(lambda x: x.expanding().mean().shift(1).fillna(average_distance)).reset_index(level=0, drop=True)


average_wind = df['wind'].mean()


df_fg['avg_prev_wind'] = df_fg.sort_values(by='game_date').groupby('kicker_player_id')['wind'] \
    .apply(lambda x: x.expanding().mean().shift(1).fillna(average_wind)).reset_index(level=0, drop=True)


average_temp = df_fg['temp'].mean()

df_fg['avg_prev_temp'] = df_fg.sort_values(by='game_date').groupby('kicker_player_id')['temp'] \
    .apply(lambda x: x.expanding().mean().shift(1).fillna(average_temp)).reset_index(level=0, drop=True)





In [355]:
len(df_fg['conditions'].unique())

28

In [356]:
import pandas as pd

# Define keywords for each weather category
rain_keywords = ['Rain', 'Showers', 'Drizzle', 'Rainy', 'Thunder', 'Storm', 'Rain and Wind', 'Rain showers', 'Light Showers']
no_rain_keywords = ['0% Chance of Rain', 'Zero Percent Chance of Rain', 'No chance of rain']
snow_keywords = ['Snow', 'Snow Showers', 'Flurries', 'Blizzard', 'Snowy', 'Snow showers']
sunny_keywords = [
    'Sunny', 'Clear', 'Fair', 'Sun', 'Sunshine', 'Sunlit', 'Bright', 
    'Cold', 'Breezy', 'Dry', 'Upper 40s to mid 30s by end of game', 'Frigid',
    'Temperatures to remain in 70s throughout the game', 'T: 66; H: 47%; WS, 9 mph', 
    'T: 66; h: 60%; W: NW 7mph'
]
cloudy_keywords = [
    'Cloudy', 'Overcast', 'Clouds', 'Fog', 'Foggy', 'Hazy', 'Misty', 'Cloud', 
    'Mostly Coudy', 'Coudy', 'Partly Clouidy', 'Partly Cloundy', 'Hazey'
]
indoor_keywords = [
    'Indoor', 'Indoors', 'N/A (Indoors)', 'Controlled Climate', 'Controlled  Climate', 
    'Controlled Climage', 'N/A Indoors Temp:', 'N/A (Indoors) Temp:', 'n/a Temp:', 'N/A Temp:'
]

def categorize_weather(condition):
    condition = condition.lower()  # Case-insensitive matching
    if any(keyword.lower() in condition for keyword in indoor_keywords):
        return 'Indoor'
    elif any(keyword.lower() in condition for keyword in no_rain_keywords):
        return 'Sunny'
    elif any(keyword.lower() in condition for keyword in snow_keywords):
        return 'Snow'
    elif any(keyword.lower() in condition for keyword in rain_keywords):
        return 'Rain'
    elif any(keyword.lower() in condition for keyword in sunny_keywords):
        return 'Sunny'
    elif any(keyword.lower() in condition for keyword in cloudy_keywords):
        return 'Cloudy'
    else:
        return 'Indoor'  # Default all other uncategorized conditions to Indoor

# Assuming df_fg is your DataFrame and 'conditions' column exists
df_fg['weather_category'] = df_fg['conditions'].apply(categorize_weather)

# Display unique conditions grouped by category
grouped_conditions = df_fg.groupby('weather_category')['conditions'].unique()
print(grouped_conditions)

df_fg.drop('conditions',axis=1, inplace=True)




weather_category
Cloudy    [Cloudy, Overcast, CLOUDY, Coudy, cloudy, Hazy, overcast, Foggy, Hazey, Fog]
Indoor                                                      [indoor, Indoors, Outdoor]
Rain                                          [Rain, Rainy, Showers, Raining, Drizzle]
Snow                                                                            [Snow]
Sunny                    [Sunny, Clear, Fair, Cold, Breezy, Dry, sunny, clear, Frigid]
Name: conditions, dtype: object


In [357]:
# # Assuming df_fg['weather_category'] has been correctly filled by previous code

# # Group by 'weather_category' and gather unique conditions for each category
# unique_conditions_by_category = df_fg.groupby('weather_category')['conditions'].unique()

# # Print the unique conditions for each weather category
# for category, conditions in unique_conditions_by_category.items():
#     print(f"Weather Category: {category}")
#     print(f"Conditions: {conditions}\n")


In [358]:
dummies = pd.get_dummies(df_fg['weather_category'], prefix='Weather')

# Join the dummy variables with the original DataFrame
df_fg = pd.concat([df_fg, dummies], axis=1)
df_fg.drop('weather_category',axis=1,inplace=True)



In [368]:
df_model = df_fg[['kick_distance','temp','wind','kick_number','weighted_accuracy','avg_prev_distance','avg_prev_wind','avg_prev_temp',
                  'Weather_Cloudy','Weather_Indoor','Weather_Rain','Weather_Snow','Weather_Sunny','result']]




In [387]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming 'df_model' is your DataFrame with features and target variable 'result'

# Split the data into features (X) and target (y)
X = df_model.drop('result', axis=1)
y = df_model['result']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27)

# Create a random forest classifier
# Create a logistic regression classifier
log_reg_model = LogisticRegression(random_state=27, max_iter=10000)  # Increase max_iter if needed for convergence

# Train the model on the training data
log_reg_model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = log_reg_model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.85


In [378]:
import tensorflow as tf

# Build the ANN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Use 'sigmoid' for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Use 'binary_crossentropy' for binary classification
              metrics=['accuracy'])

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')


In [380]:
# Train the model again
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")


Epoch 1/50
189/189 [==============================] - 0s 618us/step - loss: 0.3711 - accuracy: 0.8416 - val_loss: 0.3583 - val_accuracy: 0.8619
Epoch 2/50
189/189 [==============================] - 0s 485us/step - loss: 0.3692 - accuracy: 0.8426 - val_loss: 0.3580 - val_accuracy: 0.8606
Epoch 3/50
189/189 [==============================] - 0s 466us/step - loss: 0.3694 - accuracy: 0.8440 - val_loss: 0.3743 - val_accuracy: 0.8546
Epoch 4/50
189/189 [==============================] - 0s 469us/step - loss: 0.3706 - accuracy: 0.8430 - val_loss: 0.3584 - val_accuracy: 0.8599
Epoch 5/50
189/189 [==============================] - 0s 502us/step - loss: 0.3667 - accuracy: 0.8444 - val_loss: 0.3590 - val_accuracy: 0.8592
Epoch 6/50
189/189 [==============================] - 0s 852us/step - loss: 0.3689 - accuracy: 0.8430 - val_loss: 0.3646 - val_accuracy: 0.8579
Epoch 7/50
189/189 [==============================] - 0s 522us/step - loss: 0.3687 - accuracy: 0.8437 - val_loss: 0.3595 - val_accuracy:

In [389]:
from sklearn.linear_model import LogisticRegression

# Create a logistic regression classifier
log_reg_model = LogisticRegression(random_state=27, max_iter=1000000)  # Increase max_iter if needed for convergence

# Train the model on the training data
log_reg_model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = log_reg_model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.85
